# 텍스트 청킹 방법론 비교 실험 보고서

**작성자**: Gemini
**실험일**: 2025-11-24

## 1. 실험 개요

본 문서는 RAG(Retrieval-Augmented Generation) 시스템의 성능에 핵심적인 영향을 미치는 **텍스트 청킹(Text Chunking)**의 다양한 방법론을 비교 분석하기 위한 실험 보고서입니다.

하나의 의료 상담 데이터(JSON)를 대상으로 아래 네 가지 다른 청킹 전략을 적용하고, 그 결과를 비교하여 각 방법론의 특징과 장단점을 파악하는 것을 목표로 합니다.

### 대상 데이터
- **파일**: `data/TS_말뭉치데이터_내과/0a1fbe96-e151-11ef-a39c-00155dced605.json`

### 비교 방법론
1. **Recursive Character Text Splitting**: 재귀적 문자 분할
2. **KSS (Korean Sentence Splitter)**: 한국어 문장 단위 분할
3. **KSS Sentence Grouping**: KSS로 분리 후, 지정된 글자 수로 문장 그룹핑
4. **Semantic Chunking**: 임베딩 기반 의미 단위 분할

## 2. 공통 설정 및 라이브러리 설치

실험에 필요한 공통 라이브러리를 설치하고, 데이터 로드를 위한 헬퍼 함수와 대상 파일을 정의합니다.

In [1]:
!pip install -U -q langchain langchain-core langchain-community langchain-experimental langchain-openai openai kss sentence-transformers

In [2]:
import json
import os
from langchain_core.documents import Document
from typing import List

# --- 실험 대상 파일 정의 ---
TARGET_JSON_FILE = '/Users/test/Assign/자연어처리/rag/data/TS_말뭉치데이터_내과/0a1fbe96-e151-11ef-a39c-00155dced605.json'

def load_document_from_file(file_path):
    """지정된 단일 JSON 파일에서 Document 객체를 로드합니다."""
    try:
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)
            page_content = data.get('disease')
            if page_content and isinstance(page_content, str):
                metadata = {
                    'source': file_path,
                    'title': data.get('title', 'N/A'),
                    'department': data.get('department', 'N/A')
                }
                print(f"문서 로드 성공: {os.path.basename(file_path)}")
                return Document(page_content=page_content, metadata=metadata)
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {file_path}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생 {file_path}: {e}")
    return None

def print_chunks(chunks: List[Document]):
    """청킹된 결과를 보기 쉽게 출력합니다."""
    print(f"\n--- 총 {len(chunks)}개의 청크 생성됨 ---")
    for i, chunk in enumerate(chunks):
        chunk_len = len(chunk.page_content) if isinstance(chunk, Document) else len(chunk)
        print(f"\n--- CHUNK {i+1} (길이: {chunk_len}) ---")
        if isinstance(chunk, Document):
            print(f"[METADATA]: {chunk.metadata}")
            print(chunk.page_content)
        else:
            print(chunk)

# --- 데이터 로드 ---
doc = load_document_from_file(TARGET_JSON_FILE)

문서 로드 성공: 0a1fbe96-e151-11ef-a39c-00155dced605.json


## 3. 청킹 방법론별 실험
---

### 방법 1: Recursive Character Text Splitter

가장 일반적인 청킹 방법 중 하나입니다. 지정된 문자열(`separators`)을 기준으로 텍스트를 재귀적으로 분할합니다. 큰 덩어리에서 작은 덩어리로 나누며, 청크 크기(`chunk_size`)와 중첩 크기(`chunk_overlap`)를 유연하게 조절할 수 있습니다.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("### 방법 1: Recursive Character Text Splitter 실행 ###")

if doc:
    recursive_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n\n", "\n\n", "\n", ". ", " ", ""],
        chunk_size=1200,
        chunk_overlap=200,
        length_function=len,
    )
    
    recursive_chunks = recursive_splitter.split_documents([doc])
    print_chunks(recursive_chunks)

/opt/miniconda3/envs/rnn_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 방법 1: Recursive Character Text Splitter 실행 ###

--- 총 8개의 청크 생성됨 ---

--- CHUNK 1 (길이: 734) ---
[METADATA]: {'source': '/Users/test/Assign/자연어처리/rag/data/TS_말뭉치데이터_내과/0a1fbe96-e151-11ef-a39c-00155dced605.json', 'title': '소동물 주요 질환의 임상추론과 감별진단', 'department': '내과'}
황달 증례의 임상적 추론
황달 증례에 대한 임상적 분석
7
황달 증례의 임상적 추론
황달 증례에 대한 임상적 분석
임상 증례
황달
V는 4살의 중성화한 수컷 프렌치 불독으로 7일 동안 무감각증(apathy), 식욕저하, 구토 증상을 보였으며 지난 24시간 동안 황달 소견이 관찰되었다. 보호자는 6개월 전부터 V가 점진적인 체중 감소와 식욕저하를 보였다고 덧붙였다. 최근 약물 복용, 독소에 대한 노출과 백신 접종 이력(보통 정기적인 접종을 함에도 불구하고)은 없는 것으로 확인되었다. 환자는 최근 생활 반경을 벗어나는 여행이나 벼룩 등에 대한 노출 이력도 없다. 신체검사에서도 약간의 황달 증상을 보이는 분홍빛 점막을 제외하고는 특별한 변화는 확인되지 않았다.
사례에 적용되는 추론 방법


확보된 정보를 바탕으로 다음 2단계의 임상적 추론을 진행한다:
1. 환자의 문제를 파악한다
2. 문제의 우선순위를 설정한다
환자는 지난 7일 동안 식욕부진, 무감각증, 구토, 황달 및 만성적인 체중 감소를 보이고 있다.
식욕감소/식욕부진, 점진적인 체중 감소, 무감각증으로 시작되어 구토로 이어지고 결국 황달 증상까지 보이는 해당 환자의 경우, 만성적인 질병의 진행을 염두에 두고 임상적 특징을 조사하는 것이 타당해 보인다. 감별진단 목록에서 가장 중요한 임상 증상 또는 소견은 황달이 될 것이다. 구토 또한 중요한 증상이지만 황달과 연관되어 있거나 속발적으로 발생할 수 있으므로 우선 고려 항목에서는 제외할 수 

### 방법 2: KSS (Korean Sentence Splitter)

한국어 특성을 잘 이해하는 문장 분리 라이브러리입니다. 텍스트를 의미적으로 가장 작은 단위인 **'문장'**으로 분할합니다. 각 문장이 하나의 청크가 되므로, 정보의 최소 단위를 보존하는 데 유리하지만 청크의 길이가 매우 다양해질 수 있습니다.

In [4]:
import kss

print("### 방법 2: KSS (Korean Sentence Splitter) 실행 ###")

if doc:
    kss_chunks = []
    try:
        sentences = kss.split_sentences(doc.page_content)
        for sentence in sentences:
            kss_chunks.append(Document(page_content=sentence, metadata=doc.metadata))
        print_chunks(kss_chunks)
    except Exception as e:
        print(f"KSS 처리 중 오류 발생: {e}")

[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



### 방법 2: KSS (Korean Sentence Splitter) 실행 ###

--- 총 107개의 청크 생성됨 ---

--- CHUNK 1 (길이: 159) ---
[METADATA]: {'source': '/Users/test/Assign/자연어처리/rag/data/TS_말뭉치데이터_내과/0a1fbe96-e151-11ef-a39c-00155dced605.json', 'title': '소동물 주요 질환의 임상추론과 감별진단', 'department': '내과'}
황달 증례의 임상적 추론
황달 증례에 대한 임상적 분석
7
황달 증례의 임상적 추론
황달 증례에 대한 임상적 분석
임상 증례
황달
V는 4살의 중성화한 수컷 프렌치 불독으로 7일 동안 무감각증(apathy), 식욕저하, 구토 증상을 보였으며 지난 24시간 동안 황달 소견이 관찰되었다.

--- CHUNK 2 (길이: 44) ---
[METADATA]: {'source': '/Users/test/Assign/자연어처리/rag/data/TS_말뭉치데이터_내과/0a1fbe96-e151-11ef-a39c-00155dced605.json', 'title': '소동물 주요 질환의 임상추론과 감별진단', 'department': '내과'}
보호자는 6개월 전부터 V가 점진적인 체중 감소와 식욕저하를 보였다고 덧붙였다.

--- CHUNK 3 (길이: 66) ---
[METADATA]: {'source': '/Users/test/Assign/자연어처리/rag/data/TS_말뭉치데이터_내과/0a1fbe96-e151-11ef-a39c-00155dced605.json', 'title': '소동물 주요 질환의 임상추론과 감별진단', 'department': '내과'}
최근 약물 복용, 독소에 대한 노출과 백신 접종 이력(보통 정기적인 접종을 함에도 불구하고)은 없는 것으로 확인되었다.

--- CHUNK 4 (길이: 43) ---
[METADATA]: {'source': '/Users/test/Assign/자

### 방법 3: KSS Sentence Grouping

방법 2의 단점을 보완하는 전략입니다. 먼저 KSS를 사용해 문장 단위로 분할한 뒤, 지정된 최대 글자 수(`max_chars`)를 넘지 않는 선에서 연속된 문장들을 다시 하나의 청크로 묶습니다. 문장의 의미를 최대한 보존하면서 청크의 길이를 비교적 균일하게 유지할 수 있는 장점이 있습니다.

In [5]:
print("### 방법 3: KSS Sentence Grouping 실행 ###")

def group_sentences_by_char_limit(docs: List[Document], max_chars: int = 500) -> List[Document]:
    final_chunks = []
    for doc in docs:
        try:
            sentences = kss.split_sentences(doc.page_content)
        except Exception as e:
            print(f"KSS 처리 중 오류 발생: {e}")
            final_chunks.append(doc)
            continue
        
        current_chunk_sentences = []
        current_chunk_chars = 0
        for sentence in sentences:
            sentence_len = len(sentence)
            if current_chunk_chars + sentence_len > max_chars and current_chunk_sentences:
                chunk_text = " ".join(current_chunk_sentences)
                final_chunks.append(Document(page_content=chunk_text, metadata=doc.metadata))
                current_chunk_sentences = [sentence]
                current_chunk_chars = sentence_len
            else:
                current_chunk_sentences.append(sentence)
                current_chunk_chars += sentence_len
        
        if current_chunk_sentences:
            chunk_text = " ".join(current_chunk_sentences)
            final_chunks.append(Document(page_content=chunk_text, metadata=doc.metadata))
    return final_chunks

if doc:
    grouped_chunks = group_sentences_by_char_limit([doc], max_chars=500)
    print_chunks(grouped_chunks)

### 방법 3: KSS Sentence Grouping 실행 ###

--- 총 15개의 청크 생성됨 ---

--- CHUNK 1 (길이: 457) ---
[METADATA]: {'source': '/Users/test/Assign/자연어처리/rag/data/TS_말뭉치데이터_내과/0a1fbe96-e151-11ef-a39c-00155dced605.json', 'title': '소동물 주요 질환의 임상추론과 감별진단', 'department': '내과'}
황달 증례의 임상적 추론
황달 증례에 대한 임상적 분석
7
황달 증례의 임상적 추론
황달 증례에 대한 임상적 분석
임상 증례
황달
V는 4살의 중성화한 수컷 프렌치 불독으로 7일 동안 무감각증(apathy), 식욕저하, 구토 증상을 보였으며 지난 24시간 동안 황달 소견이 관찰되었다. 보호자는 6개월 전부터 V가 점진적인 체중 감소와 식욕저하를 보였다고 덧붙였다. 최근 약물 복용, 독소에 대한 노출과 백신 접종 이력(보통 정기적인 접종을 함에도 불구하고)은 없는 것으로 확인되었다. 환자는 최근 생활 반경을 벗어나는 여행이나 벼룩 등에 대한 노출 이력도 없다. 신체검사에서도 약간의 황달 증상을 보이는 분홍빛 점막을 제외하고는 특별한 변화는 확인되지 않았다. 사례에 적용되는 추론 방법


확보된 정보를 바탕으로 다음 2단계의 임상적 추론을 진행한다 :
1. 환자의 문제를 파악한다 2. 문제의 우선순위를 설정한다

--- CHUNK 2 (길이: 485) ---
[METADATA]: {'source': '/Users/test/Assign/자연어처리/rag/data/TS_말뭉치데이터_내과/0a1fbe96-e151-11ef-a39c-00155dced605.json', 'title': '소동물 주요 질환의 임상추론과 감별진단', 'department': '내과'}
환자는 지난 7일 동안 식욕부진, 무감각증, 구토, 황달 및 만성적인 체중 감소를 보이고 있다. 식욕감소/식욕부진, 점진적인 체중 감소, 무감각증으로 시작되어

### 방법 4: Semantic Chunker

가장 진보된 청킹 방법론 중 하나입니다. 텍스트를 임베딩하여 문장 간의 의미적 유사도를 계산하고, 의미가 크게 바뀌는 지점(breakpoint)을 찾아 청크를 분할합니다. 고정된 크기가 아닌, 문맥의 흐름에 따라 동적으로 청크를 생성하므로 정보의 의미적 일관성을 유지하는 데 가장 효과적일 수 있습니다.

**주의**: 이 방법은 `OPENAI_API_KEY` 환경 변수 설정이 필요하며, 유료 API 호출이 발생할 수 있습니다.

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings

print("### 방법 4: Semantic Chunker 실행 ###")

if 'OPENAI_API_KEY' not in os.environ or not os.environ['OPENAI_API_KEY']:
    print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("!!! 경고: OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.      !!!")
    print("!!! Semantic Chunking을 실행할 수 없습니다.                 !!!")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
else:
    if doc:
        # text-embedding-3-small 모델 사용
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile", # 또는 "standard_deviation", "interquartile"
            breakpoint_threshold_amount=95 # 분할 민감도 (값이 높을수록 더 적게 분할)
        )
        
        semantic_chunks = semantic_splitter.split_documents([doc])
        print_chunks(semantic_chunks)

## 4. 결론

위 네 가지 방법의 청킹 결과를 통해 각 전략의 특성을 확인할 수 있습니다.

- **Recursive Character Splitter**: 가장 제어하기 쉽고 빠르지만, 단어 중간이나 문장의 중간에서 잘릴 위험이 있습니다.
- **KSS**: 문장 단위로 정보를 보존하지만, 생성된 청크의 길이가 매우 불규칙하여 긴 문장이 포함된 경우 검색 효율이 떨어질 수 있습니다.
- **KSS Sentence Grouping**: KSS와 Recursive 방식의 절충안으로, 문장의 의미를 보존하면서도 청크 길이를 어느 정도 제어할 수 있어 한국어 환경에서 효과적인 대안이 될 수 있습니다.
- **Semantic Chunker**: 가장 의미론적으로 뛰어난 분할을 보여주지만, API 비용이 발생하고 처리 속도가 느리다는 단점이 있습니다.

프로젝트의 요구사항(비용, 속도, 검색 정확도 등)에 따라 적절한 청킹 전략을 선택해야 합니다.